# 1. Gradient Vanishing / Exploding
 - Gradient Vanishing : Gradient가 점차 작아지며 사라지는 것
 - Gradient Exploding : Gradient가 너무 커지는 것

# 2. Gradient Vanishing / Exploding 해결법
### (1) 간접적인 방법
 - 활성화 함수 바꾸기 (Sigmoid -> ReLU)
 - 초기화 잘하기
 - learning rate 작게 하기 : Exploding 해결법

<br>

### (2) 직접적인 방법
 - Batch Normalization

# 3. Batch Normalization 장점
 - 학습 과정 안정화
 - Regularization
 - 학습 속도 가속

# 4. Internal Covariate Shift
### (1) Covariate Shift
 - Covariate Shift는 Train set과 Test set의 분포 차이가 문제를 발생시킬 수 있다는 개념이다.
 - Covariate Shift는 입력과 출력의 분포 차이가 문제를 발생시킬 수 있다는 개념이다.
 - ex) 데이터를 학습 할 때는 실제 고양이들을 이용하고, Test 할 때 만화에 나오는 고양이를 준 다음 이게 고양이인지 확인해보라 하는 것이다. 이 예제에서는 Train set과 Test set의 분포가 2D와 3D, 사진과 그림, 색상 등 많은 부분에서 달랐다.
 
<br>

### (2) Internal Covariate Shift
 - Internal Covariate Shift가 Gradient Vanishing / Exploding의 원인이다.
 - 네트워크를 통한 학습 과정 중, 각 레이어에서 Covariate Shift가 발생하는 문제이다. 따라서 학습 전에 한 번만 정규화 한다고 해결되는 문제가 아니다!
 - 해결법 : Batch Normalization
 - ex) 고양이의 사진을 통해 최종적으로 고양이인지 아닌지 분류하는 NN 네트워크가 있다고 할 때, 
 Input인 고양이의 사진의 분포가 각 레이어를 지나며 학습되며 점차 분포가 변해 Output과 Input의 분포 차이가 발생한다.

# 5. Batch Normalization
### (1) Batch Normalization이란?
 - 각 레이어에서 분포가 변형되는 Internal Covariate Shift를 해결하기 위해 각 레이어마다 Normalization을 해준다.
 - 학습은 mini-batch 단위로 이루어지기 때문에, mini-batch마다 Normalization을 해준다는 의미에서 Batch Normalization이라고 명칭되었다.
 - Batch Normalization을 사용하면 더 좋은 정확도를 얻을 수 있다!
 
<br>

### (2) Batch Normalization 알고리즘
 - batch size의 평균, varianve를 계산한 후 Normalization을 해 준다. 이 결과 값에 Scale을 곱하고 Shift를 더해 준다.
 - 자세한 내용 : https://www.youtube.com/watch?v=HCEr5f-LfVE&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=18&t=0s
 
<br>
 
### (3) Batch Normalization 사용 방법
 - layer마다 Batch Normalization을 적용하면 되고, Activation 함수 이전에 사용하면 된다! 
 - Dropout을 사용할 때, 모델을 train 모드와 test 모드로 분할해서 사용한 것처럼, Batch Normalization을 사용할 때도 train 모드와 test 모드로 분할해서 사용해야 한다!

# 6. MNIST Batchnorm 실습

In [1]:
import torchvision.datasets as dsets # torchvision의 datasets를 불러와서 dsets로 이용
import torch
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda가 사용 가능한 상황이면 cuda를 쓰고, 아니면 CPU를 사용

# 데이터 불러오기
mnist_train = dsets.MNIST(root = "MNIST_data/", train = True, transform = transforms.ToTensor(), download = True) # MNIST 함수를 이용함. root로 경로 지정, train으로 train set 불러올지 설정, transform으로 MNIST 일반 이미지를 PyTorch가 이미지 인식하는 방법대로 읽어오게 설정. download로 root에 MNIST 데이터가 없으면 다운을 받아오게 설정.
mnist_test = dsets.MNIST(root = "MNIST_data/", train = False, transform = transforms.ToTensor(), download=True)

# 불러온 데이터 사용하기
batch_size = 128
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle=True, drop_last = True) # 데이터를 실제로 사용하기 위해서 DataLoader를 이용한다. DataLoader로 어떤 데이터를 로드 할 지 설정, batch_size로 총 60,000개의 데이터를 몇개 씩 잘라서 가져올 지 설정, shuffle로 데이터를 랜덤 순서로 불러올지 설정, drop_last로 batch_size로 짤라서 불러 올 때 맨 마지막에 남은 찌끄러기들을 사용 안 할지 설정

criterion = torch.nn.CrossEntropyLoss().to(device) # Cost. CrossEntropy에는 Softmax가 들어있다.

# 앞에서 배운 부분과 동일!
linear1 = torch.nn.Linear(784, 32, bias = True)
linear2 = torch.nn.Linear(32, 32, bias = True)
linear3 = torch.nn.Linear(32, 10, bias = True)
relu = torch.nn.ReLU()

# Batch Normalization!
bn1 = torch.nn.BatchNorm1d(32) # layer의 output이 32인 것을 Batch Normalization 할 것이다.
bn2 = torch.nn.BatchNorm1d(32) # layer의 output이 32인 것을 Batch Normalization 할 것이다.

bn_model = torch.nn.Sequential(linear1, bn1, relu, # layer마다 BN 적용! Activation 함수 이전에 사용!
                               linear2, bn2, relu, # layer마다 BN 적용! Activation 함수 이전에 사용!
                               linear3).to(device)

# 앞에서 배운 부분과 동일!
training_epochs = 15
learning_rate = 0.01
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)

for epoch in range(training_epochs):
    bn_model.train() # 필수! train 모드 사용!
 
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)

        bn_optimizer.zero_grad()
        bn_loss.backward()
        bn_optimizer.step()
